<a href="https://colab.research.google.com/github/sandoval19/Santander-challange/blob/master/Santander_CPT_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import the packages 

In [0]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import os 
import warnings
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from sklearn.decomposition import PCA
from tensorflow import keras
tf.enable_eager_execution()

Check if everything is okay

In [0]:
is_correct_tf_version = '1.14.' in tf.__version__
assert is_correct_tf_version, "Wrong tensorflow version {} installed".format(tf.__version__)

is_eager_enabled = tf.executing_eagerly()
assert is_eager_enabled,      "Tensorflow eager mode is not enabled"

#Analisis de los datos 

In [0]:
print(train_data.head())
print(test_data.head())
print("dimensiones del train set:{}, test set:{}".format(train_data.shape,test_data.shape))

In [0]:
def missing_data(data):
	"""
	Analyze the data, which its type is dataFrame to look for missing total values
	input: dataFrame 
	output: if not: Boolean, if there is missing data: dataFrame with missing values per colums
	"""
	flag = data.isna().sum().any()
	if flag:
		null_data=data.isnull().sum()
		percent= (null_data/data.isnull().count())*100

		#new data frame to return 
		mis_data= pd.concat([null_data,percent],axis=1,keys=['Total','Percent'])
		columns_type=[]
		for col in data.columns:
			dtype= str(data[col].dtype)
			columns_type.append(dtype)
		mis_data['Type'] = columns_type
		return (np.transpose(mis_data))
	else:
		return(False)

df_count_train=(missing_data(train_data))
print(df_count_train)
df_count_test=(missing_data(test_data))
print(df_count_test)

Se puede observar para cada feature no hay valores perdidos

Ahora se analizan algunos valores estadisticos de los data set: total, mean, std, min, percentils.

In [0]:
train_stats = train_data.describe()
print(train_stats)
test_stats = test_data.describe()
print(test_stats)

**Al ver la desviación estander y mean se puede inferir que los features (columnas) no estan normalizados: diferentes std y mean para cada feature.**

**Tampoco escalados: rangos de min-max considerables para cada feature.
Se considera hacer escalamiento, normalización/estandarización**

Se Analiza la distribución de datos para ambos casos target

In [0]:
sns.countplot(train_data['target'],palette='Set2')
plt.show()

#pandas acomoda de manera descendente la predominancia
print( train_data["target"].value_counts())
print("There are {}% target values with 1".format(100 * train_data["target"].value_counts()[1]/train_data.shape[0]))

Se muestrea el set de train para generar un data set mas parejo en clases

In [0]:
df_ones = train_data[train_data['target'] > 0]
print("Shape of Clase One's",df_ones.shape)
#se toman un 15% de la clase 1 
fracc=0.15
df_zeros = train_data[train_data['target'] == 0].sample(frac=fracc)
print("Shape {} of Clase Zero's with: {} '%' of the original set".format(df_zeros.shape,fracc))
#we concat both to the sampling dataframe
#if frac is used with value 1 it will return all the data but shuffled
train_data = pd.concat([df_ones, df_zeros]).sample(frac=1) #shuffling
test_data= test_data.sample(n=train_data.shape[0])
print("Shape of the new data sampled:",train_data.shape)
print("Shape of the new test_data: ", test_data.shape)
print(train_data.head())
print(train_data.tail())

sns.countplot(train_data['target'],palette='Set2')
plt.title('New data distribution after regroup')
plt.show()
print( train_data["target"].value_counts())

print("There are {}% target values with 1".format(100 * train_data["target"].value_counts()[1]/train_data.shape[0]))


Se analiza la distribución normal de las features(columns) para los dos casos de target

In [0]:
def plot_feature_distribution(df1, df2, label1, label2, features):
    """
    Grafica la probabilidad de la funcion de densidad (PDF) para ambos dataframes 
    basado en que las features estan en ambos dataframes y las etiqueta dependiendo del label
    input: dataframe1, dataframe2, string:label 1, string:label 2,lista/serie:features
    output: Nan
    """
    i = 0
    sns.set_style('whitegrid')
    plt.figure()
    fig, ax = plt.subplots(10,10,figsize=(22,22))
    for feature in features:
        i += 1
        plt.subplot(10,10,i)
        sns.distplot(df1[feature], hist=False,label=label1)
        sns.distplot(df2[feature], hist=False,label=label2)
        plt.xlabel(feature, fontsize=9)
        locs, labels = plt.xticks()
        plt.tick_params(axis='x', which='major', labelsize=6)
        plt.tick_params(axis='y', which='major', labelsize=6)
    plt.show();

In [0]:
#primeros 100 features
t0 = train_data.loc[train_data['target'] == 0]
t1 = train_data.loc[train_data['target'] == 1]
features1 = train_data.columns.values[2:102]
print(len(features1))
plot_feature_distribution(t0, t1, '0', '1', features1)
#los 100 features restantes
features2 = train_data.columns.values[102:202]
plot_feature_distribution(t0, t1, '0', '1', features2)

**Se analizar si entre los dos target alguna feature se diferencia mucho entre 1 y 0,
puede ser un factor importante que el modelo aprenda a diferenciar entre las dos clases.
Así también como features que sean muy similares, brindarían poca información para diferencias las clases**

Ahora se analiza la distribución de las features(columns) para los set de train y test

In [0]:
plot_feature_distribution(train_data,test_data,'train','test',features1)
plot_feature_distribution(train_data,test_data,'train','test',features2)

**Se puede analizar que entre train y test algunas features o son similares en distribución,
o son muy distintas.
Esto puede llevar a un criterio de selccion de features, ya que al ser distintas en train y test
el modelo aprendera un comportamiento para esa feature que nunca va a ver en el test, así clasificando 
de manera erronea.**